In [ ]:
import uproot
import numpy as np
import awkward as ak

from coffea.nanoevents import NanoEventsFactory, BaseSchema
from agc_schema import AGCSchema

In [ ]:
import json
with open('ntuples_with_tprime.json', 'r') as f:
    metadata = json.load(f)

# Function to calculate the integrated luminosity in case we couldn't access all of the files
def getLumi(nevts):
    lumi_nom = 2256.38 # total possible luminosity in /pb
    sf = float(nevts)/36619019
    return lumi_nom*sf 

# Useful function to calculate weight, pass the sample name, 
# total_events summed over files accessed and lumi calculated from function above
def getEventWeight(sample,n_total_events,lumi):
    if sample != "data": # we don't weight the real data
        xsec_weight = metadata[sample]['filter_eff']*metadata[sample]['xsec'] * lumi / (n_total_events)
    else:
        xsec_weight = 1
    return xsec_weight

In [ ]:
# define the binning for my observable (m_bjj)
obsbins=np.arange(500,2080,80)

In [ ]:
# define a function that will run over the analysis for any input file 

import time
import matplotlib.pyplot as plt

def runAnalysis(input_file,isMC=True):
    try: 
        events = NanoEventsFactory.from_root(input_file, schemaclass=AGCSchema, treepath='events').events()
    except OSError:
        time.sleep(2) # sleep for 2 seconds and try one more time
        try: 
            events = NanoEventsFactory.from_root(input_file, schemaclass=AGCSchema, treepath='events').events()
        except OSError:
            time.sleep(2) # sleep for 2 seconds just to not try EOS again but give up on this file now
            return [],[]
    
    # Object selection
    selected_electrons = events.electron[(events.electron.pt > 30) & (abs(events.electron.eta)<2.1) & (events.electron.isTight == True) & (events.electron.sip3d < 4 )]
    selected_muons = events.muon[(events.muon.pt > 30) & (abs(events.muon.eta)<2.1) & (events.muon.isTight == True) & (events.muon.sip3d < 4) & (events.muon.pfreliso04DBCorr < 0.15)]
    selected_jets = events.jet[(events.jet.corrpt > 30) & (abs(events.jet.eta)<2.4) ]
    
    # Event selection
    event_filters = ((ak.count(selected_electrons.pt, axis=1) + ak.count(selected_muons.pt, axis=1)) == 1)
    
    # Separate event selection into signal region and control region 
    event_filters_CR = event_filters &  (ak.count(selected_jets.corrpt, axis=1) >= 4) & (ak.sum(selected_jets.btag >= 0.8, axis=1) < 1)
    event_filters_SR = event_filters &  (ak.count(selected_jets.corrpt, axis=1) >= 4) & (ak.sum(selected_jets.btag >= 0.8, axis=1) >= 2)
    
    selected_events_SR = selected_jets[event_filters_SR]
    selected_events_CR = selected_jets[event_filters_CR]
    
    # calculate scale up/down 
    if isMC:
        selected_events_SR['scaleUp']   = selected_events_SR.corrptUp/selected_events_SR.corrpt
        selected_events_SR['scaleDown'] = selected_events_SR.corrptDown/selected_events_SR.corrpt
        
        selected_events_CR['scaleUp']   = selected_events_CR.corrptUp/selected_events_CR.corrpt
        selected_events_CR['scaleDown'] = selected_events_CR.corrptDown/selected_events_CR.corrpt
    
    # Calculate observable for SR (mbjj)
    trijet = ak.combinations(selected_events_SR, 3, fields=["j1", "j2", "j3"])
    j1,j2,j3 = ak.unzip(trijet)
    has_bjet = (ak.sum(j1.btag>=0.8,axis=1)+ak.sum(j2.btag>=0.8,axis=1)+ak.sum(j3.btag>=0.8,axis=1)>0)
    trijet = trijet[has_bjet]
    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3
    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
    observableSR = ak.flatten(trijet_mass).to_numpy()
     
    if isMC:
        trijet["p4Up"]   = trijet.j1.multiply(trijet.j1.scaleUp) +trijet.j2.multiply(trijet.j2.scaleUp)+trijet.j3.multiply(trijet.j3.scaleUp) 
        trijet["p4Down"] = trijet.j1.multiply(trijet.j1.scaleDown) +trijet.j2.multiply(trijet.j2.scaleDown)+trijet.j3.multiply(trijet.j3.scaleDown) 
        
        trijet_massU = trijet["p4Up"][ak.argmax(trijet.p4Up.pt, axis=1, keepdims=True)].mass
        trijet_massD = trijet["p4Down"][ak.argmax(trijet.p4Down.pt, axis=1, keepdims=True)].mass
    
        observableSRU = ak.flatten(trijet_massU).to_numpy()
        observableSRD = ak.flatten(trijet_massD).to_numpy()
    
    
    # Calculate observable for CR (mjjj)
    trijet = ak.combinations(selected_events_CR, 3, fields=["j1", "j2", "j3"])
    j1,j2,j3 = ak.unzip(trijet)
    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3
    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
    observableCR = ak.flatten(trijet_mass).to_numpy()
    
    if isMC:
        trijet["p4Up"]   = trijet.j1.multiply(trijet.j1.scaleUp) +trijet.j2.multiply(trijet.j2.scaleUp)+trijet.j3.multiply(trijet.j3.scaleUp) 
        trijet["p4Down"] = trijet.j1.multiply(trijet.j1.scaleDown) +trijet.j2.multiply(trijet.j2.scaleDown)+trijet.j3.multiply(trijet.j3.scaleDown) 
        
        trijet_massU = trijet["p4Up"][ak.argmax(trijet.p4Up.pt, axis=1, keepdims=True)].mass
        trijet_massD = trijet["p4Down"][ak.argmax(trijet.p4Down.pt, axis=1, keepdims=True)].mass
    
        observableCRU = ak.flatten(trijet_massU).to_numpy()
        observableCRD = ak.flatten(trijet_massD).to_numpy()
    
    # create counts in bins 
    counts_bin_edgesSR  = plt.hist(observableSR,bins=obsbins)
    countsSR, bin_edgesSR = counts_bin_edgesSR[0],counts_bin_edgesSR[1]
    
    if isMC : 
        countsSRU_bin_edgesSR  = plt.hist(observableSRU,bins=obsbins)
        countsSRU, bin_edgesSR = countsSRU_bin_edgesSR[0],countsSRU_bin_edgesSR[1]
    
        countsSRD_bin_edgesSR  = plt.hist(observableSRD,bins=obsbins)
        countsSRD, bin_edgesSR = countsSRD_bin_edgesSR[0],countsSRD_bin_edgesSR[1]
    
    plt.clf()
    
    
    counts_bin_edgesCR  = plt.hist(observableCR,bins=obsbins)
    countsCR, bin_edgesCR = counts_bin_edgesCR[0],counts_bin_edgesCR[1]

   
    if isMC: 
        countsCRU_bin_edgesCR  = plt.hist(observableCRU,bins=obsbins)
        countsCRU, bin_edgesCR = countsCRU_bin_edgesCR[0],countsCRU_bin_edgesCR[1]
    
        countsCRD_bin_edgesCR  = plt.hist(observableCRD,bins=obsbins)
        countsCRD, bin_edgesCR = countsCRD_bin_edgesCR[0],countsCRD_bin_edgesCR[1]
    
    plt.clf()
    
    # return the counts 
    # return the counts 
    if isMC: 
        return {'nominal':np.array(countsSR),'jesUp':np.array(countsSRU),'jesDown':np.array(countsSRD)},{'nominal':np.array(countsCR),'jesUp':np.array(countsCRU),'jesDown':np.array(countsCRD)}
    else : return {'nominal':np.array(countsSR)},{'nominal':np.array(countsCR)}


In [ ]:
from hist2df import * 

samples=[
    'data'
    ,'ttbar'
    ,'single_atop_t_chan'
    ,'single_top_t_chan'
    ,'single_top_tW'
    ,'wjets'
    ,'tprime'
]


n_sim_samples = len(samples)-1

# first run over the data. We want to keep track of the total number of events 
# that get accessed so that we can modify the integrated luminosity if necessary 

print('data')
countsSR = {'nominal':np.array([0. for i in range(len(obsbins)-1)])}
countsCR = {'nominal':np.array([0. for i in range(len(obsbins)-1)])}
total_events = 0

MAX_FILES = -1 # if a negative number, then we don't set a maximum on the number of files. 

for j,p in enumerate(metadata['data']['nominal']['files']): 
    
    # for each file, we add the counts from these events
    if (MAX_FILES > 0 and j > MAX_FILES): continue
    print(j,p['path'])

    event_countsSR,event_countsCR = runAnalysis(p['path'],isMC=False)
    if len(event_countsSR):
        countsSR['nominal'] += event_countsSR['nominal']
        countsCR['nominal'] += event_countsCR['nominal']
        total_events+=p['nevts'] 
 
    else : print("skipped file (XRootD error)")
    

intlumi = getLumi(total_events)
print("Data total events accessed", total_events, " -> %g /pb"%intlumi )
                 
# convert to dataframe 
dfs   = histogramToDataframe(countsSR['nominal'],"signalregion",'data')
dfsCR = histogramToDataframe(countsCR['nominal'],"controlregion",'data')
dfs = pd.concat([dfs,dfsCR],ignore_index=True)
                
# Now we run over the simulated samples. Again, we should keep track of the number of events  
# that get accessed so that we can calculate the right event_weight

for i,sample in enumerate(samples[1:]): 
    if sample=='data': continue # shouldn't need this 
    print(sample)
    
     # make sure to zero out the counts for each sample 
    countsSR = {
         'nominal':np.array([0. for i in range(len(obsbins)-1)])
        ,'jesUp':np.array([0. for i in range(len(obsbins)-1)])
        ,'jesDown':np.array([0. for i in range(len(obsbins)-1)])
    }
    countsCR = {
         'nominal':np.array([0. for i in range(len(obsbins)-1)])
        ,'jesUp':np.array([0. for i in range(len(obsbins)-1)])
        ,'jesDown':np.array([0. for i in range(len(obsbins)-1)])
    }
    total_sim_events = 0
    
    for j,p in enumerate(metadata[sample]['nominal']['files']): 
        
        if (MAX_FILES > 0 and j > MAX_FILES): continue
        print(j,p['path'])
        # for each file, we add the counts from these events
        event_countsSR,event_countsCR = runAnalysis(p['path'],isMC=True)
        if len(event_countsSR):
            countsSR['nominal'] += event_countsSR['nominal']
            countsCR['nominal'] += event_countsCR['nominal']
            
            countsSR['jesUp'] += event_countsSR['jesUp']
            countsCR['jesUp'] += event_countsCR['jesUp']
            
            countsSR['jesDown'] += event_countsSR['jesDown']
            countsCR['jesDown'] += event_countsCR['jesDown']
            total_sim_events+=p['nevts'] 
           
        else : print("skipped file (XRootD error)")
        
            
    event_weight = getEventWeight(sample,total_sim_events,intlumi)
    print("Total events processed (%s):"%(sample), total_sim_events, 
          "XSec = ",metadata[sample]['xsec'], 
          "Filter efficiency = ", metadata[sample]['filter_eff'], 
          "Event weight = ", event_weight)
        
    # now that we have the counts, we convert to a dataframe 
    dfs_sampleSRN = histogramToDataframe(countsSR['nominal'],"signalregion",sample)
    dfs_sampleCRN = histogramToDataframe(countsCR['nominal'],"controlregion",sample)
    
    dfs_sampleSRU = histogramToDataframe(countsSR['jesUp'],"signalregion",sample,sys='jesUp')
    dfs_sampleCRU = histogramToDataframe(countsCR['jesUp'],"controlregion",sample,sys='jesUp')
    
    dfs_sampleSRD = histogramToDataframe(countsSR['jesDown'],"signalregion",sample,sys='jesDown')
    dfs_sampleCRD = histogramToDataframe(countsCR['jesDown'],"controlregion",sample,sys='jesDown')
    
    dfs_sample = pd.concat([dfs_sampleSRN,dfs_sampleCRN,dfs_sampleSRU,dfs_sampleCRU,dfs_sampleSRD,dfs_sampleCRD])
    
    # apply weight to columns
    dfs_sample['sum_w']*=event_weight
    dfs_sample['sum_ww']*=event_weight*event_weight
    
    # add the new samples into our dataframe for signal and control regions
    dfs = pd.concat([dfs,dfs_sample],ignore_index=True)

In [ ]:
plt.rcParams.update({'font.size': 18})

order_sim = samples[1:-1] # don't include the signal in the stack
order_sim.reverse() # plot stack in reverse 
n_sim_samples = len(order_sim)

# need to make list structures for this to work in stacked mode 
bin_centres = np.array(obsbins)+(obsbins[1]-obsbins[0])/2
bins_list = [obsbins for b in range(n_sim_samples)]

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(14,6))

def mkPlot(axis,region,obsname,title):
    
    samples_stack = [dfs[ (dfs['process']==s) & (dfs['systematic']=='nominal') & (dfs['channel']==region)]['sum_w'].to_list() for s in order_sim]
    labels = order_sim


    axis.hist(bins_list,bins=obsbins,weights=samples_stack,label=labels,stacked=True,density=False)
    axis.errorbar(bin_centres,dfs[ (dfs['process']=='data') & (dfs['systematic']=='nominal') & (dfs['channel']==region)]['sum_w']
             ,yerr     =(dfs[ (dfs['process']=='data') & (dfs['systematic']=='nominal') & (dfs['channel']==region)]['sum_ww'])**0.5
             ,label='data (%.2f $fb^{-1}$)'%(intlumi/1000)
             ,marker='o'
             ,markersize=3.2
             ,color='black'
             ,linestyle="none")
    
    axis.hist(obsbins,bins=obsbins
         ,weights=dfs[ (dfs['process']=='tprime') & (dfs['systematic']=='nominal') & (dfs['channel']==region)]['sum_w']
         ,color='cyan'
         ,histtype='step'
         ,linewidth=3
         ,label='tprime - $m_{T}=1.2$ TeV'
        )
    axis.set_xlabel(obsname) #"$m_{bjj}$ (GeV)")
    axis.set_ylabel("Events")
    #axis.set_yscale("log")
    axis.set_title(title)
    axis.legend()

mkPlot(ax1,"signalregion","$m_{bjj}$ (GeV)","4j2b")
mkPlot(ax2,"controlregion","$m_{jjj}$ (GeV)","4j0b")

fig.tight_layout()
fig.savefig("TPrimesearch_dists.png")

In [ ]:
# sort the dataframe to avoid warnings later
dfs = dfs.sort_values(by=['channel','process','systematic','bin'])

# Finally save histograms to a csv file
dfs.to_csv('tprime_1200GeV_search_allregions_mbjj.csv',index=False)